# 🎬 Rotten Tomatoes Movies – ML Analysis**Dataset**: 16,638 movies with critic & audience ratings, genres, runtime, and metadata.| # | Task | Type | Target ||---|------|------|--------|| 1 | Tomatometer Status Prediction | Multi-class Classification | Rotten / Fresh / Certified Fresh || 2 | Audience Rating Prediction | Regression | audience_rating || 3 | Tomatometer Rating Prediction | Regression | tomatometer_rating || 4 | Movie Clustering | Unsupervised | K-Means on movie profiles |

## 1 · Imports

In [ ]:
import warnings, os, pathlibwarnings.filterwarnings("ignore")import numpy as npimport pandas as pdimport matplotlibmatplotlib.use("Agg")import matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.model_selection import (train_test_split, GridSearchCV,                                     RandomizedSearchCV, cross_val_score,                                     learning_curve, StratifiedKFold)from sklearn.preprocessing import StandardScaler, LabelEncoderfrom sklearn.metrics import (accuracy_score, f1_score, classification_report,                             confusion_matrix, mean_squared_error, r2_score,                             mean_absolute_error, silhouette_score)from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNetfrom sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressorfrom sklearn.ensemble import (RandomForestClassifier, RandomForestRegressor,                              GradientBoostingClassifier, GradientBoostingRegressor,                              AdaBoostClassifier, VotingClassifier, StackingClassifier)from sklearn.svm import SVC, SVRfrom sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressorfrom sklearn.neural_network import MLPClassifierfrom sklearn.naive_bayes import GaussianNBfrom sklearn.cluster import KMeansfrom xgboost import XGBClassifierimport jinja2, base64from io import BytesIOSEED = 42PLOT_DIR = pathlib.Path("outputs/plots")PLOT_DIR.mkdir(parents=True, exist_ok=True)sns.set_theme(style="whitegrid", palette="viridis")print("✅ Imports OK")

## 2 · Load & Prepare Data

In [ ]:
df_raw = pd.read_csv("Rotten Tomatoes Movies.csv")print(f"Raw shape: {df_raw.shape}")print(f"Missing values:\n{df_raw.isnull().sum()}")# ── Clean rating typos ──df_raw["rating"] = df_raw["rating"].str.strip().str.rstrip(")")print(f"\nRatings after cleanup: {df_raw['rating'].unique()}")# ── Parse dates ──df_raw["in_theaters_date"] = pd.to_datetime(df_raw["in_theaters_date"], errors="coerce")df_raw["on_streaming_date"] = pd.to_datetime(df_raw["on_streaming_date"], errors="coerce")# ── Feature engineering ──df = df_raw.copy()# Theater year / monthdf["theater_year"] = df["in_theaters_date"].dt.yeardf["theater_month"] = df["in_theaters_date"].dt.month# Genre count & top-10 binary flagsdf["genre_count"] = df["genre"].fillna("").apply(lambda x: len([g for g in x.split(",") if g.strip()]))all_genres = df["genre"].fillna("").str.split(",\s*").explode().str.strip()top_genres = all_genres[all_genres != ""].value_counts().head(10).index.tolist()for g in top_genres:    df[f"genre_{g.lower().replace(' & ', '_').replace(' ', '_')}"] = (        df["genre"].fillna("").str.contains(g, case=False).astype(int)    )print(f"Top genres encoded: {top_genres}")# Cast sizedf["cast_size"] = df["cast"].fillna("").apply(lambda x: len(x.split(",")) if x else 0)# Has critics consensusdf["has_consensus"] = df["critics_consensus"].notna().astype(int)# Title lengthdf["title_length"] = df["movie_title"].str.len()df["title_word_count"] = df["movie_title"].str.split().str.len()# Rating ordinal encoderating_map = {"G": 0, "PG": 1, "PG-13": 2, "R": 3, "NC17": 4, "NR": 5}df["rating_enc"] = df["rating"].map(rating_map).fillna(5).astype(int)# Log audience countdf["log_audience_count"] = np.log1p(df["audience_count"].fillna(0))# Days to streamingdf["days_to_streaming"] = (df["on_streaming_date"] - df["in_theaters_date"]).dt.days# Drop rows with critical missing valuesdf = df.dropna(subset=["runtime_in_minutes", "audience_rating", "tomatometer_rating"])print(f"\nClean shape: {df.shape}")df.head(3)

## 3 · Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))# 1 – Tomatometer status distributiondf["tomatometer_status"].value_counts().plot.bar(ax=axes[0, 0], color=sns.color_palette("viridis", 3))axes[0, 0].set_title("Tomatometer Status Distribution")axes[0, 0].set_xlabel("Status"); axes[0, 0].set_ylabel("Count")axes[0, 0].tick_params(axis="x", rotation=0)# 2 – Tomatometer vs Audience rating scatteraxes[0, 1].scatter(df["tomatometer_rating"], df["audience_rating"], alpha=0.15, s=5, c="teal")axes[0, 1].set_title("Tomatometer vs Audience Rating")axes[0, 1].set_xlabel("Tomatometer Rating"); axes[0, 1].set_ylabel("Audience Rating")# 3 – Runtime distributiondf["runtime_in_minutes"].clip(upper=250).hist(bins=50, ax=axes[0, 2], color="steelblue", edgecolor="white")axes[0, 2].set_title("Runtime Distribution (capped 250 min)")axes[0, 2].set_xlabel("Minutes"); axes[0, 2].set_ylabel("Count")# 4 – Rating distributiondf["rating"].value_counts().plot.bar(ax=axes[1, 0], color=sns.color_palette("mako", df["rating"].nunique()))axes[1, 0].set_title("MPAA Rating Distribution")axes[1, 0].set_xlabel("Rating"); axes[1, 0].set_ylabel("Count")axes[1, 0].tick_params(axis="x", rotation=0)# 5 – Movies per yearyear_counts = df["theater_year"].dropna().astype(int).value_counts().sort_index()year_counts = year_counts[year_counts.index >= 1970]axes[1, 1].plot(year_counts.index, year_counts.values, color="darkgreen", linewidth=2)axes[1, 1].set_title("Movies Per Year (since 1970)")axes[1, 1].set_xlabel("Year"); axes[1, 1].set_ylabel("Count")# 6 – Genre frequency (top 10)genre_freq = all_genres[all_genres != ""].value_counts().head(10)genre_freq.plot.barh(ax=axes[1, 2], color=sns.color_palette("rocket", 10))axes[1, 2].set_title("Top 10 Genres")axes[1, 2].set_xlabel("Count"); axes[1, 2].invert_yaxis()plt.tight_layout()plt.savefig(PLOT_DIR / "eda_overview.png", dpi=150, bbox_inches="tight")plt.show()print("✅ EDA plots saved")# Correlation heatmapnum_cols = ["runtime_in_minutes", "tomatometer_rating", "tomatometer_count",            "audience_rating", "audience_count", "genre_count", "cast_size",            "has_consensus", "title_length", "rating_enc", "log_audience_count"]fig, ax = plt.subplots(figsize=(12, 9))sns.heatmap(df[num_cols].corr(), annot=True, fmt=".2f", cmap="coolwarm", ax=ax,            linewidths=0.5, vmin=-1, vmax=1)ax.set_title("Correlation Heatmap")plt.tight_layout()plt.savefig(PLOT_DIR / "correlation_heatmap.png", dpi=150, bbox_inches="tight")plt.show()print("✅ Correlation heatmap saved")

## 4 · Prepare Features for Classification

In [ ]:
# Feature columnsfeature_cols = [    "runtime_in_minutes", "tomatometer_count", "audience_rating", "audience_count",    "genre_count", "cast_size", "has_consensus", "title_length", "title_word_count",    "rating_enc", "log_audience_count"] + [c for c in df.columns if c.startswith("genre_")]# Also add theater_year, theater_month, days_to_streamingfor col in ["theater_year", "theater_month", "days_to_streaming"]:    if col in df.columns:        feature_cols.append(col)# Drop rows with NaN in featuresdf_clf = df.dropna(subset=feature_cols + ["tomatometer_status"]).copy()print(f"Classification dataset: {df_clf.shape}")X_clf = df_clf[feature_cols].fillna(0).valuesle_status = LabelEncoder()y_clf = le_status.fit_transform(df_clf["tomatometer_status"])print(f"Classes: {le_status.classes_}")print(f"Class distribution: {np.bincount(y_clf)}")X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(    X_clf, y_clf, test_size=0.2, random_state=SEED, stratify=y_clf)scaler_c = StandardScaler()X_train_cs = scaler_c.fit_transform(X_train_c)X_test_cs = scaler_c.transform(X_test_c)print(f"Train: {X_train_cs.shape}, Test: {X_test_cs.shape}")

## 5 · Tomatometer Status Classification (10 Models)

In [ ]:
classifiers = {    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=SEED),    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=7),    "Decision Tree": DecisionTreeClassifier(random_state=SEED),    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1),    "Gradient Boosting": GradientBoostingClassifier(n_estimators=200, random_state=SEED),    "AdaBoost": AdaBoostClassifier(n_estimators=150, random_state=SEED),    "XGBoost": XGBClassifier(n_estimators=200, use_label_encoder=False,                              eval_metric="mlogloss", random_state=SEED, n_jobs=-1),    "SVM": SVC(kernel="rbf", probability=True, random_state=SEED),    "Naive Bayes": GaussianNB(),    "MLP": MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=400, random_state=SEED),}clf_results = {}for name, model in classifiers.items():    print(f"Training {name}...", end=" ")    model.fit(X_train_cs, y_train_c)    y_pred = model.predict(X_test_cs)    acc = accuracy_score(y_test_c, y_pred)    f1 = f1_score(y_test_c, y_pred, average="weighted")    clf_results[name] = {"accuracy": acc, "f1": f1, "model": model, "y_pred": y_pred}    print(f"Acc={acc:.4f}  F1={f1:.4f}")best_clf_name = max(clf_results, key=lambda k: clf_results[k]["f1"])print(f"\n🏆 Best classifier: {best_clf_name} (F1={clf_results[best_clf_name]['f1']:.4f})")# Bar chartfig, ax = plt.subplots(figsize=(12, 6))names = list(clf_results.keys())accs = [clf_results[n]["accuracy"] for n in names]f1s = [clf_results[n]["f1"] for n in names]x = np.arange(len(names))ax.bar(x - 0.2, accs, 0.4, label="Accuracy", color="steelblue")ax.bar(x + 0.2, f1s, 0.4, label="F1 (weighted)", color="coral")ax.set_xticks(x); ax.set_xticklabels(names, rotation=45, ha="right")ax.set_ylim(0, 1); ax.set_title("Classification Model Comparison")ax.legend(); plt.tight_layout()plt.savefig(PLOT_DIR / "model_comparison.png", dpi=150, bbox_inches="tight")plt.show()print("✅ Model comparison saved")

## 6 · Audience Rating Regression

In [ ]:
# Use tomatometer_rating + other features to predict audience_ratingreg_features = [    "runtime_in_minutes", "tomatometer_rating", "tomatometer_count",    "genre_count", "cast_size", "has_consensus", "title_length", "title_word_count",    "rating_enc", "log_audience_count"] + [c for c in df.columns if c.startswith("genre_")]for col in ["theater_year", "theater_month", "days_to_streaming"]:    if col in df.columns:        reg_features.append(col)df_reg = df.dropna(subset=reg_features + ["audience_rating"]).copy()X_reg = df_reg[reg_features].fillna(0).valuesy_reg = df_reg["audience_rating"].valuesX_train_r, X_test_r, y_train_r, y_test_r = train_test_split(    X_reg, y_reg, test_size=0.2, random_state=SEED)scaler_r = StandardScaler()X_train_rs = scaler_r.fit_transform(X_train_r)X_test_rs = scaler_r.transform(X_test_r)regressors = {    "Ridge": Ridge(alpha=1.0),    "Lasso": Lasso(alpha=0.1, max_iter=2000),    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=2000),    "Decision Tree": DecisionTreeRegressor(random_state=SEED),    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=SEED, n_jobs=-1),    "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, random_state=SEED),}reg_results = {}for name, model in regressors.items():    print(f"Training {name}...", end=" ")    model.fit(X_train_rs, y_train_r)    y_pred = model.predict(X_test_rs)    r2 = r2_score(y_test_r, y_pred)    rmse = np.sqrt(mean_squared_error(y_test_r, y_pred))    mae = mean_absolute_error(y_test_r, y_pred)    reg_results[name] = {"r2": r2, "rmse": rmse, "mae": mae, "model": model, "y_pred": y_pred}    print(f"R²={r2:.4f}  RMSE={rmse:.2f}  MAE={mae:.2f}")best_reg_name = max(reg_results, key=lambda k: reg_results[k]["r2"])print(f"\n🏆 Best regressor (audience rating): {best_reg_name} (R²={reg_results[best_reg_name]['r2']:.4f})")# Actual vs Predicted scatterfig, axes = plt.subplots(2, 3, figsize=(18, 10))for ax, (name, res) in zip(axes.flat, reg_results.items()):    ax.scatter(y_test_r, res["y_pred"], alpha=0.2, s=8, c="teal")    ax.plot([0, 100], [0, 100], "r--", linewidth=1.5)    ax.set_title(f"{name}\nR²={res['r2']:.4f}"); ax.set_xlabel("Actual"); ax.set_ylabel("Predicted")plt.suptitle("Audience Rating Regression – Actual vs Predicted", fontsize=14, y=1.01)plt.tight_layout()plt.savefig(PLOT_DIR / "audience_regression_results.png", dpi=150, bbox_inches="tight")plt.show()print("✅ Audience regression plots saved")

## 7 · Tomatometer Rating Regression

In [ ]:
# Use audience_rating + other features to predict tomatometer_ratingtom_features = [    "runtime_in_minutes", "audience_rating", "tomatometer_count",    "genre_count", "cast_size", "has_consensus", "title_length", "title_word_count",    "rating_enc", "log_audience_count"] + [c for c in df.columns if c.startswith("genre_")]for col in ["theater_year", "theater_month", "days_to_streaming"]:    if col in df.columns:        tom_features.append(col)df_tom = df.dropna(subset=tom_features + ["tomatometer_rating"]).copy()X_tom = df_tom[tom_features].fillna(0).valuesy_tom = df_tom["tomatometer_rating"].valuesX_train_t, X_test_t, y_train_t, y_test_t = train_test_split(    X_tom, y_tom, test_size=0.2, random_state=SEED)scaler_t = StandardScaler()X_train_ts = scaler_t.fit_transform(X_train_t)X_test_ts = scaler_t.transform(X_test_t)tom_regressors = {    "Ridge": Ridge(alpha=1.0),    "Lasso": Lasso(alpha=0.1, max_iter=2000),    "Decision Tree": DecisionTreeRegressor(random_state=SEED),    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=SEED, n_jobs=-1),    "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, random_state=SEED),}tom_results = {}for name, model in tom_regressors.items():    print(f"Training {name}...", end=" ")    model.fit(X_train_ts, y_train_t)    y_pred = model.predict(X_test_ts)    r2 = r2_score(y_test_t, y_pred)    rmse = np.sqrt(mean_squared_error(y_test_t, y_pred))    mae = mean_absolute_error(y_test_t, y_pred)    tom_results[name] = {"r2": r2, "rmse": rmse, "mae": mae, "model": model, "y_pred": y_pred}    print(f"R²={r2:.4f}  RMSE={rmse:.2f}  MAE={mae:.2f}")best_tom_name = max(tom_results, key=lambda k: tom_results[k]["r2"])print(f"\n🏆 Best regressor (tomatometer): {best_tom_name} (R²={tom_results[best_tom_name]['r2']:.4f})")# Feature importance (best model)best_tom_model = tom_results[best_tom_name]["model"]if hasattr(best_tom_model, "feature_importances_"):    imp = best_tom_model.feature_importances_    idx = np.argsort(imp)[-15:]    fig, ax = plt.subplots(figsize=(10, 7))    ax.barh(np.array(tom_features)[idx], imp[idx], color="darkcyan")    ax.set_title(f"Tomatometer Regression – Feature Importance ({best_tom_name})")    ax.set_xlabel("Importance")    plt.tight_layout()    plt.savefig(PLOT_DIR / "tomatometer_feature_importance.png", dpi=150, bbox_inches="tight")    plt.show()    print("✅ Tomatometer feature importance saved")

## 8 · Movie Clustering

In [ ]:
clust_features = [    "runtime_in_minutes", "tomatometer_rating", "tomatometer_count",    "audience_rating", "genre_count", "cast_size", "rating_enc",    "log_audience_count", "has_consensus"]df_clust = df.dropna(subset=clust_features).copy()X_clust = df_clust[clust_features].fillna(0).valuesscaler_cl = StandardScaler()X_clust_s = scaler_cl.fit_transform(X_clust)# Elbow + SilhouetteK_range = range(2, 11)inertias, sils = [], []for k in K_range:    km = KMeans(n_clusters=k, random_state=SEED, n_init=10)    labels = km.fit_predict(X_clust_s)    inertias.append(km.inertia_)    sils.append(silhouette_score(X_clust_s, labels, sample_size=5000, random_state=SEED))fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))ax1.plot(list(K_range), inertias, "bo-"); ax1.set_title("Elbow Method")ax1.set_xlabel("k"); ax1.set_ylabel("Inertia")ax2.plot(list(K_range), sils, "rs-"); ax2.set_title("Silhouette Score")ax2.set_xlabel("k"); ax2.set_ylabel("Score")plt.tight_layout()plt.savefig(PLOT_DIR / "elbow_silhouette.png", dpi=150, bbox_inches="tight")plt.show()best_k = list(K_range)[np.argmax(sils)]print(f"Best k={best_k}, silhouette={max(sils):.4f}")km_final = KMeans(n_clusters=best_k, random_state=SEED, n_init=10)df_clust["cluster"] = km_final.fit_predict(X_clust_s)# Cluster profilescluster_profile = df_clust.groupby("cluster")[clust_features].mean()print("\nCluster profiles:")print(cluster_profile.round(2).to_string())# Radar-style comparisonfig, ax = plt.subplots(figsize=(12, 6))cluster_profile_norm = (cluster_profile - cluster_profile.min()) / (cluster_profile.max() - cluster_profile.min() + 1e-9)cluster_profile_norm.T.plot(kind="bar", ax=ax, colormap="viridis")ax.set_title(f"Cluster Profiles (k={best_k})")ax.set_ylabel("Normalized value"); ax.set_xlabel("Feature")ax.legend(title="Cluster", bbox_to_anchor=(1.05, 1)); ax.tick_params(axis="x", rotation=45)plt.tight_layout()plt.savefig(PLOT_DIR / "clustering_results.png", dpi=150, bbox_inches="tight")plt.show()print("✅ Clustering plots saved")

## 9 · Hyperparameter Tuning

In [ ]:
TUNE_SIZE = min(8000, len(X_train_cs))X_tune = X_train_cs[:TUNE_SIZE]y_tune = y_train_c[:TUNE_SIZE]# GridSearchCV – Random Forestprint("GridSearchCV on Random Forest...")rf_grid = {    "n_estimators": [100, 200],    "max_depth": [10, 20, None],    "min_samples_split": [2, 5],}gs_rf = GridSearchCV(RandomForestClassifier(random_state=SEED, n_jobs=-1),                     rf_grid, cv=3, scoring="f1_weighted", n_jobs=-1)gs_rf.fit(X_tune, y_tune)print(f"  Best params: {gs_rf.best_params_}")print(f"  Best CV F1:  {gs_rf.best_score_:.4f}")# RandomizedSearchCV – Gradient Boostingprint("\nRandomizedSearchCV on Gradient Boosting...")gb_dist = {    "n_estimators": [100, 200, 300],    "max_depth": [3, 5, 7],    "learning_rate": [0.01, 0.05, 0.1, 0.2],    "subsample": [0.7, 0.8, 1.0],}rs_gb = RandomizedSearchCV(GradientBoostingClassifier(random_state=SEED),                           gb_dist, n_iter=20, cv=3, scoring="f1_weighted",                           random_state=SEED, n_jobs=-1)rs_gb.fit(X_tune, y_tune)print(f"  Best params: {rs_gb.best_params_}")print(f"  Best CV F1:  {rs_gb.best_score_:.4f}")# Evaluate tuned models on test setfor label, model in [("Tuned RF (Grid)", gs_rf.best_estimator_),                     ("Tuned GB (Random)", rs_gb.best_estimator_)]:    y_pred = model.predict(X_test_cs)    acc = accuracy_score(y_test_c, y_pred)    f1 = f1_score(y_test_c, y_pred, average="weighted")    clf_results[label] = {"accuracy": acc, "f1": f1, "model": model, "y_pred": y_pred}    print(f"  {label}: Acc={acc:.4f}  F1={f1:.4f}")print("\n✅ Hyperparameter tuning complete")

## 10 · Cross-Validation, Feature Importance, Confusion Matrices & Learning Curves

In [ ]:
CV_SIZE = min(8000, len(X_clf))X_cv = X_clf[:CV_SIZE]y_cv = y_clf[:CV_SIZE]scaler_cv = StandardScaler()X_cvs = scaler_cv.fit_transform(X_cv)# ── 5-fold CV ──cv_models = {    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=SEED),    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1),    "Gradient Boosting": GradientBoostingClassifier(n_estimators=200, random_state=SEED),    "XGBoost": XGBClassifier(n_estimators=200, eval_metric="mlogloss", random_state=SEED, n_jobs=-1),}cv_scores = {}for name, model in cv_models.items():    scores = cross_val_score(model, X_cvs, y_cv, cv=5, scoring="f1_weighted", n_jobs=-1)    cv_scores[name] = scores    print(f"{name}: mean F1={scores.mean():.4f} ± {scores.std():.4f}")fig, ax = plt.subplots(figsize=(10, 5))ax.boxplot(cv_scores.values(), labels=cv_scores.keys())ax.set_title("5-Fold Cross-Validation F1 Scores")ax.set_ylabel("F1 (weighted)"); ax.tick_params(axis="x", rotation=20)plt.tight_layout()plt.savefig(PLOT_DIR / "cv_comparison.png", dpi=150, bbox_inches="tight")plt.show()# ── Feature importance (best classification model) ──best_clf_model = clf_results[best_clf_name]["model"]if hasattr(best_clf_model, "feature_importances_"):    imp = best_clf_model.feature_importances_    idx = np.argsort(imp)[-15:]    fig, ax = plt.subplots(figsize=(10, 7))    ax.barh(np.array(feature_cols)[idx], imp[idx], color="coral")    ax.set_title(f"Feature Importance – {best_clf_name}")    ax.set_xlabel("Importance")    plt.tight_layout()    plt.savefig(PLOT_DIR / "feature_importance.png", dpi=150, bbox_inches="tight")    plt.show()# ── Confusion matrices (top 4) ──top4 = sorted(clf_results, key=lambda k: clf_results[k]["f1"], reverse=True)[:4]fig, axes = plt.subplots(1, 4, figsize=(22, 5))for ax, name in zip(axes, top4):    cm = confusion_matrix(y_test_c, clf_results[name]["y_pred"])    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax,                xticklabels=le_status.classes_, yticklabels=le_status.classes_)    ax.set_title(f"{name}\nF1={clf_results[name]['f1']:.4f}")    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")plt.suptitle("Confusion Matrices – Top 4 Models", fontsize=14, y=1.02)plt.tight_layout()plt.savefig(PLOT_DIR / "confusion_matrices.png", dpi=150, bbox_inches="tight")plt.show()# ── Learning curves ──LC_SIZE = min(6000, len(X_clf))X_lc = X_clf[:LC_SIZE]y_lc = y_clf[:LC_SIZE]scaler_lc = StandardScaler()X_lcs = scaler_lc.fit_transform(X_lc)fig, axes = plt.subplots(1, 2, figsize=(14, 5))for ax, (name, model) in zip(axes, [("Random Forest", RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)),                                     ("Gradient Boosting", GradientBoostingClassifier(n_estimators=100, random_state=SEED))]):    train_sizes, train_scores, val_scores = learning_curve(        model, X_lcs, y_lc, cv=3, n_jobs=-1,        train_sizes=np.linspace(0.1, 1.0, 8), scoring="f1_weighted"    )    ax.plot(train_sizes, train_scores.mean(axis=1), "o-", label="Train")    ax.plot(train_sizes, val_scores.mean(axis=1), "s-", label="Validation")    ax.set_title(f"Learning Curve – {name}")    ax.set_xlabel("Training Size"); ax.set_ylabel("F1 (weighted)")    ax.legend(); ax.grid(True)plt.tight_layout()plt.savefig(PLOT_DIR / "learning_curves.png", dpi=150, bbox_inches="tight")plt.show()print("✅ CV, feature importance, confusion matrices & learning curves saved")

## 11 · Voting & Stacking Ensembles

In [ ]:
# Voting Classifierprint("Training Voting Classifier...")voting = VotingClassifier(    estimators=[        ("rf", RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1)),        ("gb", GradientBoostingClassifier(n_estimators=200, random_state=SEED)),        ("xgb", XGBClassifier(n_estimators=200, eval_metric="mlogloss", random_state=SEED, n_jobs=-1)),    ],    voting="soft")voting.fit(X_train_cs, y_train_c)y_pred_v = voting.predict(X_test_cs)acc_v = accuracy_score(y_test_c, y_pred_v)f1_v = f1_score(y_test_c, y_pred_v, average="weighted")clf_results["Voting Ensemble"] = {"accuracy": acc_v, "f1": f1_v, "model": voting, "y_pred": y_pred_v}print(f"  Voting: Acc={acc_v:.4f}  F1={f1_v:.4f}")# Stacking Classifierprint("Training Stacking Classifier...")stacking = StackingClassifier(    estimators=[        ("rf", RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1)),        ("gb", GradientBoostingClassifier(n_estimators=200, random_state=SEED)),        ("xgb", XGBClassifier(n_estimators=200, eval_metric="mlogloss", random_state=SEED, n_jobs=-1)),    ],    final_estimator=LogisticRegression(max_iter=1000, random_state=SEED),    cv=3, n_jobs=-1)stacking.fit(X_train_cs, y_train_c)y_pred_s = stacking.predict(X_test_cs)acc_s = accuracy_score(y_test_c, y_pred_s)f1_s = f1_score(y_test_c, y_pred_s, average="weighted")clf_results["Stacking Ensemble"] = {"accuracy": acc_s, "f1": f1_s, "model": stacking, "y_pred": y_pred_s}print(f"  Stacking: Acc={acc_s:.4f}  F1={f1_s:.4f}")# Final rankingprint("\n" + "="*60)print("FINAL MODEL RANKING (Classification)")print("="*60)ranking = sorted(clf_results.items(), key=lambda x: x[1]["f1"], reverse=True)for i, (name, res) in enumerate(ranking, 1):    print(f"  {i:>2}. {name:<25s} Acc={res['accuracy']:.4f}  F1={res['f1']:.4f}")best_overall = ranking[0][0]print(f"\n🏆 Best overall: {best_overall} (F1={clf_results[best_overall]['f1']:.4f})")

## 12 · Generate HTML Report

In [ ]:
def img_to_base64(path):    with open(path, "rb") as f:        return base64.b64encode(f.read()).decode()images = {}for p in sorted(PLOT_DIR.glob("*.png")):    images[p.stem] = img_to_base64(p)TEMPLATE = """<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1"><title>🎬 Rotten Tomatoes Movies – ML Report</title><style>:root{--bg:#0f172a;--card:#1e293b;--accent:#06b6d4;--text:#e2e8f0;--muted:#94a3b8}*{margin:0;padding:0;box-sizing:border-box}body{background:var(--bg);color:var(--text);font-family:'Segoe UI',system-ui,sans-serif;padding:2rem}h1{text-align:center;font-size:2.2rem;margin-bottom:.4rem;color:var(--accent)}.subtitle{text-align:center;color:var(--muted);margin-bottom:2rem}.card{background:var(--card);border-radius:12px;padding:1.5rem;margin-bottom:1.5rem;box-shadow:0 4px 24px #0004}.card h2{color:var(--accent);margin-bottom:1rem;font-size:1.3rem}table{width:100%;border-collapse:collapse;margin:1rem 0}th,td{padding:.55rem .8rem;text-align:left;border-bottom:1px solid #334155}th{color:var(--accent);font-size:.85rem;text-transform:uppercase}tr:hover{background:#ffffff08}.best{background:#06b6d415;font-weight:700}img{width:100%;border-radius:8px;margin:.8rem 0}.grid2{display:grid;grid-template-columns:1fr 1fr;gap:1.2rem}@media(max-width:800px){.grid2{grid-template-columns:1fr}}.tag{display:inline-block;padding:2px 10px;border-radius:6px;font-size:.82rem;background:#06b6d422;color:var(--accent);margin:2px}</style></head><body><h1>🎬 Rotten Tomatoes Movies – ML Report</h1><p class="subtitle">16,638 movies · 17 features · 4 ML tasks</p><div class="card"><h2>📊 Exploratory Data Analysis</h2><div class="grid2"><img src="data:image/png;base64,{{images.eda_overview}}" alt="EDA Overview"><img src="data:image/png;base64,{{images.correlation_heatmap}}" alt="Correlation Heatmap"></div></div><div class="card"><h2>🎯 Task 1 – Tomatometer Status Classification</h2><p style="margin-bottom:.8rem">Predict whether a movie is <span class="tag">Rotten</span> <span class="tag">Fresh</span> <span class="tag">Certified Fresh</span></p><table><tr><th>#</th><th>Model</th><th>Accuracy</th><th>F1 (weighted)</th></tr>{% for name, res in clf_ranking %}<tr{% if loop.first %} class="best"{% endif %}><td>{{loop.index}}</td><td>{{name}}</td><td>{{"{:.4f}".format(res.accuracy)}}</td><td>{{"{:.4f}".format(res.f1)}}</td></tr>{% endfor %}</table><img src="data:image/png;base64,{{images.model_comparison}}" alt="Model Comparison"></div><div class="card"><h2>🎬 Task 2 – Audience Rating Regression</h2><table><tr><th>#</th><th>Model</th><th>R²</th><th>RMSE</th><th>MAE</th></tr>{% for name, res in reg_ranking %}<tr{% if loop.first %} class="best"{% endif %}><td>{{loop.index}}</td><td>{{name}}</td><td>{{"{:.4f}".format(res.r2)}}</td><td>{{"{:.2f}".format(res.rmse)}}</td><td>{{"{:.2f}".format(res.mae)}}</td></tr>{% endfor %}</table><img src="data:image/png;base64,{{images.audience_regression_results}}" alt="Audience Regression"></div><div class="card"><h2>🍅 Task 3 – Tomatometer Rating Regression</h2><table><tr><th>#</th><th>Model</th><th>R²</th><th>RMSE</th><th>MAE</th></tr>{% for name, res in tom_ranking %}<tr{% if loop.first %} class="best"{% endif %}><td>{{loop.index}}</td><td>{{name}}</td><td>{{"{:.4f}".format(res.r2)}}</td><td>{{"{:.2f}".format(res.rmse)}}</td><td>{{"{:.2f}".format(res.mae)}}</td></tr>{% endfor %}</table>{% if images.tomatometer_feature_importance %}<img src="data:image/png;base64,{{images.tomatometer_feature_importance}}" alt="Tomatometer Feature Importance">{% endif %}</div><div class="card"><h2>🔬 Task 4 – Movie Clustering</h2><p>Best k={{best_k}}, Silhouette={{"{:.4f}".format(best_sil)}}</p><div class="grid2"><img src="data:image/png;base64,{{images.elbow_silhouette}}" alt="Elbow & Silhouette"><img src="data:image/png;base64,{{images.clustering_results}}" alt="Clustering Results"></div></div><div class="card"><h2>🔧 Hyperparameter Tuning & Cross-Validation</h2><div class="grid2"><img src="data:image/png;base64,{{images.cv_comparison}}" alt="CV Comparison">{% if images.feature_importance %}<img src="data:image/png;base64,{{images.feature_importance}}" alt="Feature Importance">{% endif %}</div><img src="data:image/png;base64,{{images.confusion_matrices}}" alt="Confusion Matrices"><img src="data:image/png;base64,{{images.learning_curves}}" alt="Learning Curves"></div></body></html>"""from types import SimpleNamespaceclf_ranking = [(n, SimpleNamespace(**{k: v for k, v in r.items() if k != "model" and k != "y_pred"}))               for n, r in sorted(clf_results.items(), key=lambda x: x[1]["f1"], reverse=True)]reg_ranking = [(n, SimpleNamespace(**{k: v for k, v in r.items() if k != "model" and k != "y_pred"}))               for n, r in sorted(reg_results.items(), key=lambda x: x[1]["r2"], reverse=True)]tom_ranking = [(n, SimpleNamespace(**{k: v for k, v in r.items() if k != "model" and k != "y_pred"}))               for n, r in sorted(tom_results.items(), key=lambda x: x[1]["r2"], reverse=True)]html = jinja2.Template(TEMPLATE).render(    images=images,    clf_ranking=clf_ranking,    reg_ranking=reg_ranking,    tom_ranking=tom_ranking,    best_k=best_k,    best_sil=max(sils),)out_path = pathlib.Path("outputs/movies_ml_report.html")out_path.write_text(html)print(f"✅ HTML Report generated: {out_path}")print(f"   File size: {out_path.stat().st_size / 1024:.1f} KB")print(f"   Embedded images: {len(images)}")